In [6]:
## --- PARAMETER ---
CONTEST = "W401" # Contest name
CONTEST_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-401" # API URL

PAGE = 257 # how many page gonna fetch 

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://leetcode.com/contest/',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

In [2]:
import csv
import glob
import json
import os
import sys
from tqdm import tqdm
import urllib.request

# create folder
try:
    os.mkdir(CONTEST)
    os.mkdir(CONTEST+"/Q3")
    os.mkdir(CONTEST+"/Q4")
    os.mkdir(CONTEST+"/Q3/0001-1000")
    os.mkdir(CONTEST+"/Q3/1001-2000")
    os.mkdir(CONTEST+"/Q3/2001-3000")
    os.mkdir(CONTEST+"/Q3/3001-4000")
    os.mkdir(CONTEST+"/Q3/4001-5000")
    os.mkdir(CONTEST+"/Q3/5001-6000")
    os.mkdir(CONTEST+"/Q3/6001-7000")
    os.mkdir(CONTEST+"/Q3/7001-8000")
    os.mkdir(CONTEST+"/Q4/0001-1000")
    os.mkdir(CONTEST+"/Q4/1001-2000")
    os.mkdir(CONTEST+"/Q4/2001-3000")
    os.mkdir(CONTEST+"/Q4/3001-4000")
    os.mkdir(CONTEST+"/Q4/4001-5000")
    os.mkdir(CONTEST+"/Q4/5001-6000")
    os.mkdir(CONTEST+"/Q4/6001-7000")
    os.mkdir(CONTEST+"/Q4/7001-8000")
except:
    pass

In [7]:
## get question_id
Q = []

for _ in range(10):
    try:
        req = urllib.request.Request(CONTEST_URL, headers=headers)
        r = urllib.request.urlopen(req).read()
        response = json.loads(r)
    except:
        pass
    else:
        break
else:
    print("http request error:", CONTEST_URL)
    sys.exit()
    
for question in response["questions"]:
    Q.append(str(question["question_id"]))
    print(question["question_id"], question["credit"], question["title"])

3450 3 Find the Child Who Has the Ball After K Seconds
3422 4 Find the N-th Value After K Seconds
3442 5 Maximum Total Reward Using Operations I
3443 7 Maximum Total Reward Using Operations II


In [ ]:
## create the csv to record user name and submission_id
with open(CONTEST+"/"+CONTEST+".csv", "w") as f:
    f.write("user, Q1, Q2, Q3, Q4\n");
    
    for pagination in tqdm(range(1, PAGE+1)):
        for _ in range(10):
            try:
                req = urllib.request.Request(CONTEST_URL+"?pagination=" + str(pagination) +"&region=global", headers=headers)
                r = urllib.request.urlopen(req).read()
                response = json.loads(r)
            except:
                pass
            else:
                break
        else:
            print("http request error:", CONTEST_URL+"?pagination=" + str(pagination) +"&region=global")
            sys.exit()

        for i in range(25):            
            f.write(response["total_rank"][i]["user_slug"])
            
            for j, Q_id in enumerate(Q):
                if Q_id not in response["submissions"][i]:
                    submission_id = ""
                else:   
                    submission_id = str(response["submissions"][i][Q_id]["submission_id"])
    
                f.write("," + submission_id)

            f.write("\n")

In [8]:
## --- PARAMETER ---
## download code
Q = 3 # i.e. 4 means Q4

def lang_ext(lang):
    match lang:
        case "cpp":
            return "cpp"
        case "python3" | "python":
            return "py"
        case "java":
            return "java"
        case _:
            return lang

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))

    for rank, row in enumerate(tqdm(rows)):
        if rank == 0:
            continue
            
        folder = str(rank//1000)+"001-"+str(rank//1000+1)+"000/"
        
        if row[Q] and not glob.glob(CONTEST+"/Q"+str(Q)+"/"+folder+str(rank)+".*"):
            submission_id = row[Q]
            
            for _ in range(20):
                try:
                    req = urllib.request.Request("https://leetcode.com/api/submissions/" + submission_id, headers=headers)
                    r = urllib.request.urlopen(req).read()
                    response = json.loads(r)
                except:
                    pass
                else:
                    with open(CONTEST+"/Q"+str(Q)+"/"+folder+str(rank)+"."+lang_ext(response["lang"]), "w", encoding="utf-8") as f2:
                        try:
                            f2.write(response["code"])
                        except:
                            print(submission_id)
                            sys.exit()
                    break
            else:
                print("file missed", rank, submission_id)

print("finished!!!")

100%|██████████████████████████████████████████████████████████████████████████████| 6426/6426 [01:45<00:00, 60.99it/s]

finished!!!
